In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
import os

### Define dataset path

In [ ]:
dataset_path =r'C:\Users\kshit\Downloads\archive (4)\Dataset\Train'  

In [ ]:
if not os.path.exists(dataset_path):
    print(f"Dataset path '{dataset_path}' does not exist.")
else:
    print(f"Dataset path '{dataset_path}' is valid.")

### Training the Model

In [ ]:
base_model = MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights='imagenet'

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=x)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
train_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)
batch_size = 8
train_generator = train_datagen.flow_from_directory(
    dataset_path, target_size=(128, 128), batch_size=batch_size, class_mode='binary', subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    dataset_path, target_size=(128, 128), batch_size=batch_size, class_mode='binary', subset='validation'
)

In [ ]:
checkpoint = ModelCheckpoint('best_deepfake_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')
epochs = 5
model.fit(train_generator, validation_data=validation_generator, 
          epochs=epochs, steps_per_epoch=train_generator.samples // batch_size, 
          validation_steps=validation_generator.samples // batch_size, 
          callbacks=[checkpoint])

### Saving the Model

In [ ]:
model.load_weights('best_deepfake_model.h5')